This notebook is used to stream data from dandi, and then acessing it in the brainscore scope.

In [2]:

import pandas as pd
from pynwb import NWBHDF5IO
import os 
import numpy as np
import numpy as np
import xarray as xr

from nwbwidgets import nwb2widget
from brainio.assemblies import NeuronRecordingAssembly

############### Open NWB File #################################################
###############################################################################

inventory   = '/braintree/home/aliya277/inventory/'
all_files = os.listdir(inventory)
folder = all_files[5]
path = os.path.join(inventory, folder)
io = NWBHDF5IO(os.path.join(path, f"{folder}.nwb"), "r") 
nwbfile = io.read()

for string in nwbfile.session_description.split(', '):
    if string.startswith('ON/OFF'):
        on_off = string.split(":")[-1]
        try:
            on_off = on_off.split('/')
        except: pass
        on_off = [eval(i) for i in on_off]
    if string.startswith('Visual Deg'):
        vis_deg = eval(string.split(':')[-1])

spikeTimes = nwbfile.units['spike_times'][:]
try:    
    psth = nwbfile.scratch['psth'][:] #[stimuli x reps x timebins x channels]
    [start_time_ms, stop_time_ms, tb_ms] = nwbfile.scratch['psth meta'][:] 
    try:
        # stimulus presentation times, i.e. start of each trial
        stim_start_time_ms = nwbfile.intervals['trials']['start_time'][:] 
        stim_stop_time_ms  = nwbfile.intervals['trials']['stop_time'][:]
        assert 'ms' == nwbfile.intervals['trials']['unit'][:][0]
    except:
        stim_start_time_ms = None
        stim_stop_time_ms = None
       
except: psth = None


# io.close()


In [3]:
nwb2widget(nwbfile)

In [ ]:
def load_responses(data_dir, stimuli):
    data_dir = data_dir / 'database'
    assert os.path.isdir(data_dir)
    psth = np.load(data_dir / 'solo.rsvp.bold5000.experiment_psth.npy')  # Shaped images x repetitions x time_bins x channels

    # Compute firing rate for given time bins
    timebins = [[70, 170], [170, 270], [50, 100], [100, 150], [150, 200], [200, 250], [70, 270]]
    photodiode_delay = 30  # Delay recorded on photodiode is ~30ms
    timebase = np.arange(-100, 381, 10)  # PSTH from -100ms to 380ms relative to stimulus onset
    assert len(timebase) == psth.shape[2]
    rate = np.empty((len(timebins), psth.shape[0], psth.shape[1], psth.shape[3]))
    for idx, tb in enumerate(timebins):
        t_cols = np.where((timebase >= (tb[0] + photodiode_delay)) & (timebase < (tb[1] + photodiode_delay)))[0]
        rate[idx] = np.mean(psth[:, :, t_cols, :], axis=2)  # Shaped time bins x images x repetitions x channels

    assembly = xr.DataArray(rate,
                            coords={'repetition': ('repetition', list(range(rate.shape[2]))),
                                    'time_bin_id': ('time_bin', list(range(rate.shape[0]))),
                                    'time_bin_start': ('time_bin', [x[0] for x in timebins]),
                                    'time_bin_stop': ('time_bin', [x[1] for x in timebins])},
                            dims=['time_bin', 'image', 'repetition', 'neuroid'])

    # Add neuroid related meta data
    neuroid_meta = pd.DataFrame(json.load(open(data_dir.parent / 'array-metadata' / 'mapping.json')))
    for column_name, column_data in neuroid_meta.iteritems():
        assembly = assembly.assign_coords(**{f'{column_name}': ('neuroid', list(column_data.values))})

    # Add stimulus related meta data
    for column_name, column_data in stimuli.iteritems():
        assembly = assembly.assign_coords(**{f'{column_name}': ('image', list(column_data.values))})

    # Collapse dimensions 'image' and 'repetitions' into a single 'presentation' dimension
    assembly = assembly.stack(presentation=('image', 'repetition')).reset_index('presentation')
    assembly = assembly.drop('image')
    assembly = NeuronRecordingAssembly(assembly)

    # Filter noisy electrodes
    psth = np.load(data_dir / 'solo.rsvp.bold5000.normalizer_psth.npy')
    t_cols = np.where((timebase >= (70 + photodiode_delay)) & (timebase < (170 + photodiode_delay)))[0]
    rate = np.mean(psth[:, :, t_cols, :], axis=2)
    normalizer_assembly = xr.DataArray(rate,
                                       coords={'repetition': ('repetition', list(range(rate.shape[1]))),
                                               'image_id': ('image', list(range(rate.shape[0]))),
                                               'id': ('image', list(range(rate.shape[0])))},
                                       dims=['image', 'repetition', 'neuroid'])
    for column_name, column_data in neuroid_meta.iteritems():
        normalizer_assembly = normalizer_assembly.assign_coords(
            **{f'{column_name}': ('neuroid', list(column_data.values))})
    normalizer_assembly = normalizer_assembly.stack(presentation=('image', 'repetition')).reset_index('presentation')
    normalizer_assembly = normalizer_assembly.drop('image')
    normalizer_assembly = normalizer_assembly.transpose('presentation', 'neuroid')
    normalizer_assembly = NeuronRecordingAssembly(normalizer_assembly)

    filtered_assembly = filter_neuroids(normalizer_assembly, 0.7)
    assembly = assembly.sel(neuroid=np.isin(assembly.neuroid_id, filtered_assembly.neuroid_id))
    assembly = assembly.transpose('presentation', 'neuroid', 'time_bin')

    # Add other experiment and data processing related info
    assembly.attrs['image_size_degree'] = 8
    assembly.attrs['stim_on_time_ms'] = 100

    return assembly

In [107]:
# Adjusted from: brainio_contrib/mkgu_packaging/dicarlo/sanghavi/sanghavijozwik2020.py load_responses

timebins = np.arange(start_time_ms, stop_time_ms, tb_ms)
assembly = xr.DataArray(psth,
                        coords={'repetition': ('repetition', list(range(psth.shape[1]))),
                                'time_bin_id': ('time_bin', list(range(psth.shape[2]))),
                                'time_bin_start': ('time_bin', [x for x in timebins]),
                                'time_bin_stop': ('time_bin', [x+tb_ms for x in timebins])
                                },
                        dims=['image', 'repetition', 'time_bin', 'neuroid'])
 # Collapse dimensions 'image' and 'repetitions' into a single 'presentation' dimension
assembly = assembly.stack(presentation=('image', 'repetition')).reset_index('presentation')
assembly = assembly.drop('image')
assembly = NeuronRecordingAssembly(assembly)
assembly = assembly.transpose('presentation', 'neuroid', 'time_bin')

# Add other experiment and data processing related info
assembly.attrs['image_size_degree'] = vis_deg
assembly.attrs['stim_on_time_ms']   = on_off[0]

assembly


<xarray.NeuronRecordingAssembly (presentation: 1440, neuroid: 192, time_bin: 30)>
array([[[ 0.,  1.,  2., ...,  0.,  0.,  1.],
        [ 0.,  1.,  0., ...,  0.,  0.,  0.],
        [ 2.,  1.,  0., ...,  0.,  0.,  0.],
        ...,
        [ 1.,  0.,  0., ...,  0.,  0.,  2.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  1., ...,  0.,  0.,  0.]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  1.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ...,
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       [[ 1.,  0.,  0., ...,  0.,  0.,  1.],
        [ 0.,  1.,  0., ...,  0.,  0.,  2.],
        [ 0.,  1.,  0., ...,  0.,  3.,  0.],
        ...,
...
        ...,
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  1.,  0.,  0.]],

       [[ 0.,  0.,  0., ...,  1.,  0.,  1.],
        [ 0.,  0.,  1., ...,  1.,  1.,  0.],
        [ 1.,  1.,  0., ...,  0.,  0.,  2.],
        ...,
        [ 0.,  1.,  1., ...,  0.,  0.,  1.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]])
Coordinates:
  * time_bin        (time_bin) MultiIndex
  - time_bin_id     (time_bin) int64 0 1 2 3 4 5 6 7 ... 22 23 24 25 26 27 28 29
  - time_bin_start  (time_bin) float64 0.0 10.0 20.0 30.0 ... 270.0 280.0 290.0
  - time_bin_stop   (time_bin) float64 10.0 20.0 30.0 40.0 ... 280.0 290.0 300.0
  * presentation    (presentation) int64 0 1 2 3 4 5 6 ... 29 30 31 32 33 34 35
Dimensions without coordinates: neuroid
Attributes:
    image_size_degree:  8
    stim_on_time_ms:    100